# Motivational Example

Let's start our course with a motivational example demonstrating a **simple end-to-end Machine Learning project** from start to finish to show you what a typical data science workflow looks like.

The example is complex on purpose as it introduces topics from almost all upcoming lectures:

- Intro to the environment and tools + Python basics
- Understanding, extracting, sourcing, and processing data
- What is AI/ML & Basic concepts in AI/ML & AI project and workflow
- (Zoo of) ML techniques (algorithms) & Training and improving ML models (ML Pipelines)
- Evaluating, comparing, and selecting ML models

At the end of the course, you will have a deeper understanding and some practical experience with those topics.

## The Problem

We are helping a client to build a mobile app for field researchers that helps them identify and track flower species. With a new app version, the client wants to automate the identification part somewhat.

At first, the client wants to assess the feasibility of this potential feature by building a POC. Luckily, the client can provide a reasonable [dataset](https://archive.ics.uci.edu/ml/datasets/iris) consisting of **four measurements** (sepal length, sepal width, petal length, and petal width) and **identity** (Iris-Setosa, Iris-Versicolour, Iris-Virginica) per sample.

![An example of Iris plant](../../static/images/petal_sepal.jpeg "An example of Iris plant")

> ✏️ The example is inspired by {cite}`rhiever`. Some sections are reduced or extended compare to the original example.

TODO: load dataset

TODO: EDA

TODO: Solving classification problem

TODO: Exercises

## Resources

```{bibliography}
:filter: docname in docnames
```